# TPD/iTPD implementation test 

In [2]:
from tpc import tp
from tpc.utils import mat_diff, ppoly2mat
import numpy as np

In [3]:
from copy import deepcopy

In [4]:
# Random hermit matrix
n = 5
N = int(2**n)
A = np.random.rand(N, N)
B = np.random.rand(N, N)
C = np.matrix(A + 1j* B)
H = 0.5*(C + C.H)

## TPD algorithm

In [5]:
# Canonical matrix
canonical = (tp.tpd(deepcopy(H)).real)

In [6]:
canonical

matrix([[ 5.44555095e-01,  6.02989910e-01,  3.87073723e-01, ...,
          4.21958047e-01,  4.19412093e-01,  5.00974270e-01],
        [-3.35484755e-02,  4.63258799e-03, -8.66116457e-03, ...,
         -1.17563344e-02,  3.66342626e-02, -1.04050009e-02],
        [-5.77824182e-02, -2.45945537e-03, -1.90482889e-02, ...,
         -5.05748751e-02, -1.85940682e-02,  2.21826094e-02],
        ...,
        [-4.48801671e-02, -3.86988267e-02,  1.48600073e-01, ...,
          7.97223287e-04,  7.67660013e-02, -6.65307687e-02],
        [-3.41855978e-02, -9.67426829e-02,  4.86579824e-02, ...,
          1.06407620e-01,  6.83721358e-02, -5.20215381e-02],
        [-7.86543963e-05,  2.13919787e-02, -6.76479695e-02, ...,
         -5.40316675e-02, -8.94000781e-03,  1.87324102e-02]])

## Canonical matrix to Pauli poly

$M$: Canonical matrix, result of TPD algorithm.
$$M_{i,j} = w_{nz, nx} $$
$$(i,j) = (nz, nz^\wedge nx)$$
$$(nz, nx) = (i, i^\wedge j)$$

In [7]:
Paulistring = ["IIIZ", "XYZI", "XXXX"]

In [10]:
ppoly = tp.pauli_basis2ppoly(canonical)

In [11]:
ppoly[:5], ppoly[-5:]

([('IIIII', 0.5445550951150779),
  ('IIIIX', 0.6029899100075405),
  ('IIIXI', 0.3870737225128674),
  ('IIIXX', 0.46490462838675406),
  ('IIXII', 0.5885352585793704)],
 [('ZZYZZ', -0.0803786496743522),
  ('ZZZYY', 0.04873110223217517),
  ('ZZZYZ', -0.054031667487650356),
  ('ZZZZY', -0.008940007810318215),
  ('ZZZZZ', 0.01873241024523649)])

In [12]:
pstrs = ppoly[:5]

In [18]:
indexes = [tp.pstr2ij_code(p[0]) for p in pstrs]

In [21]:
indexes

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]

In [19]:
N = 5
mat = np.zeros((2**N, 2**N))

In [20]:
for index, w in zip(indexes, pstrs[:, 1]):
    mat[index] = w

TypeError: list indices must be integers or slices, not tuple

## Pauli polynomial to canonical matrix

$$\sum_{nz, nx} \omega_{nz, nx} P_{nz, nx} \rightarrow M$$

In [23]:
canonical = tp.ppoly2canonical(ppoly)
canonical

matrix([[ 0.56153029+0.j,  0.45604713+0.j,  0.50851166+0.j, ...,
          0.41970604+0.j,  0.47387409+0.j,  0.46898219+0.j],
        [-0.0155733 +0.j, -0.01956309+0.j, -0.01252462+0.j, ...,
         -0.01219589+0.j, -0.04804467+0.j, -0.00874808+0.j],
        [ 0.09226451+0.j,  0.03240614+0.j, -0.1073068 +0.j, ...,
          0.04117718+0.j, -0.01437803+0.j,  0.05191796+0.j],
        ...,
        [ 0.01569611+0.j, -0.02012842+0.j,  0.01197734+0.j, ...,
         -0.08435024+0.j,  0.05659452+0.j, -0.0120251 +0.j],
        [ 0.08208257+0.j, -0.06444761+0.j,  0.09406771+0.j, ...,
         -0.00853974+0.j,  0.01713914+0.j, -0.05005177+0.j],
        [ 0.00927137+0.j, -0.07101529+0.j,  0.12513197+0.j, ...,
         -0.08088744+0.j,  0.03420327+0.j, -0.0098746 +0.j]])

In [ ]:
H_res = tp.itpd(ppoly)
H_res_naive = ppoly2mat(ppoly) # Naive tensor product

## Test the restored Hamiltonian

$$D = H - H_{res}$$
$$\sqrt{\langle D | D \rangle}$$

In [27]:
mat_diff(H, H_res), mat_diff(H, H_res_naive), mat_diff(H_res, H_res_naive)

((5.148747239520243e-16+0j),
 (2.320133686802434-6.356972294260158e-19j),
 (2.320133686802434-7.71489363849964e-19j))

## Effective Term method

In [ ]:
H_eff = tp.itpd_eff(ppoly) # Avoid unecessary calculation.

In [15]:
mat_diff(H, H_eff)

(5.267182256956205e-16+0j)